In [1]:
import pandas as pd
import json
from tqdm import tqdm
import math

from return_nearby_amenities import search_nearby_amenities, valid_amenities

In [2]:
listings = pd.read_csv('./data/processed/20231116/processed.csv')
listings.head()

,id,name,url,price,num_bedroom,num_bathroom,cost_psf,address,road_name,building,...,latitude,longitude,floor_area,land_area,walk_destination,walk_distance_m,walk_time_mins,lease_duration,completion,type
0,24243973,Affinity At Serangoon,https://www.propertyguru.com.sg/listing/for-sa...,820000,1.0,1.0,1729.96,32 SERANGOON NORTH AVENUE 1 AFFINITY AT SERANG...,SERANGOON NORTH AVENUE 1,AFFINITY AT SERANGOON,...,1.365970,103.874442,474,NaN,CR9 Serangoon North MRT,590.0,8.0,99,NaN,Condominium
1,24685653,Parc Komo,https://www.propertyguru.com.sg/listing/for-sa...,1499999,2.0,2.0,2080.44,961 UPPER CHANGI ROAD NORTH PARC KOMO SINGAPOR...,UPPER CHANGI ROAD NORTH,PARC KOMO,...,1.360923,103.969804,721,NaN,NaN,NaN,NaN,Freehold,NaN,Condominium
2,24567031,Sentosa Cove,https://www.propertyguru.com.sg/listing/for-sa...,36800000,7.0,6.0,3808.74,OCEAN DRIVE,OCEAN DRIVE,NaN,...,1.249586,103.841621,11200,9662.0,NaN,NaN,NaN,99,2011.0,Detached House
3,24567141,Sentosa Cove,https://www.propertyguru.com.sg/listing/for-sa...,27500000,7.0,6.0,3709.70,COVE WAY,COVE WAY,NaN,...,1.241634,103.839074,9480,7413.0,NaN,NaN,NaN,99,2011.0,Bungalow House
4,24705902,The Landmark,https://www.propertyguru.com.sg/listing/for-sa...,2954507,4.0,3.0,2589.40,173 CHIN SWEE ROAD THE LANDMARK SINGAPORE 169878,CHIN SWEE ROAD,THE LANDMARK,...,1.286163,103.839554,1141,NaN,NE4 Chinatown MRT,500.0,7.0,99,NaN,Condominium


In [5]:
amenities = {
    'property_id': [],
    'amenity_type': [],
    'amenity_name': [],
    'latitude': [],
    'longitude': [],
    'distance_km': [],
}

for amenity in valid_amenities:
    print(amenity)
    listings[f'has_{amenity}'] = False
    for i, row in tqdm(listings.iterrows()):
        if math.isnan(row['latitude']):
            continue
        
        results = search_nearby_amenities(
            lat1=row['latitude'],
            long1=row['longitude'],
            type=amenity,
            threshold=3
        )

        listings.at[i, f'has_{amenity}'] = (len(result)>0)
        for result in results:
            amenities['property_id'].append(row['id'])
            amenities['amenity_type'].append(amenity)
            amenities['amenity_name'].append(result['name'])
            amenities['latitude'].append(result['lat'])
            amenities['longitude'].append(result['long'])
            amenities['distance_km'].append(result['distance_km'])

disability_service


48729it [00:09, 5116.62it/s]


sport_facility


48729it [00:03, 14122.29it/s]


hawker


...

48729it [02:09, 375.88it/s]


npark


48729it [00:22, 2152.77it/s]


primary_school


48729it [00:09, 5282.80it/s]


In [8]:
amenities_df = pd.DataFrame(amenities)
amenities_df['latitude'] = amenities_df['latitude'].astype(float)
amenities_df['longitude'] = amenities_df['longitude'].astype(float)
amenities_df['distance_km'] = amenities_df['distance_km'].astype(float)
amenities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23748654 entries, 0 to 23748653
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   property_id   int64  
 1   amenity_type  object 
 2   amenity_name  object 
 3   latitude      float64
 4   longitude     float64
 5   distance_km   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 1.1+ GB


In [10]:
amenities_df.to_csv('./data/processed/20231116/property_amenities.csv', index=False)

In [11]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48729 entries, 0 to 48728
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      48729 non-null  int64  
 1   name                    48729 non-null  object 
 2   url                     48729 non-null  object 
 3   price                   48729 non-null  object 
 4   num_bedroom             48304 non-null  float64
 5   num_bathroom            48001 non-null  float64
 6   cost_psf                47955 non-null  float64
 7   address                 37597 non-null  object 
 8   road_name               37547 non-null  object 
 9   building                30414 non-null  object 
 10  postal_code             33697 non-null  float64
 11  latitude                37597 non-null  float64
 12  longitude               37597 non-null  float64
 13  floor_area              48729 non-null  int64  
 14  land_area               9166 non-null 

In [12]:
listings.to_csv('./data/processed/20231116/processed.csv', index=False)

In [15]:
import pyproj

geodesic = pyproj.Geod(ellps='WGS84')
fwd_azimuth,back_azimuth,distance = geodesic.inv(
    lats1=1.2781251110441687, 
    lons1=103.621789825727, 
    lats2=1.3562087840892174,
    lons2=104.0272222394134,
)
distance

45939.3442883239